In [1]:
import pandas as pd
import numpy as np
import math
from scipy import signal

In [2]:
xls1 = pd.ExcelFile("../Data/sub1int1.xlsx")
in_df1 = pd.read_excel(xls1, 'Sheet2')

xls2 = pd.ExcelFile("../Data/sub2int1.xlsx")
in_df2 = pd.read_excel(xls2, 'Sheet1')

xls3 = pd.ExcelFile("../Data/sub3int1.xlsx")
in_df3 = pd.read_excel(xls3, 'Sheet1')

xls4 = pd.ExcelFile("../Data/sub4int1.xlsx")
in_df4 = pd.read_excel(xls4, 'Sheet1')

out_df1 = pd.read_csv("../data/Outputs/modified_labels1.csv")
out_df2 = pd.read_csv("../data/Outputs/modified_labels2.csv")
out_df3 = pd.read_csv("../data/Outputs/modified_labels3.csv")
out_df4 = pd.read_csv("../data/Outputs/modified_labels4.csv")

In [3]:
in_df = in_df4
out_df = out_df4

In [4]:
in_df.to_csv("intermediate.csv", index = False)
in_df = pd.read_csv("intermediate.csv")

In [5]:
in_df

,C,x-mm,y-mm,z-mm,vx,vy,vz,sesno,img1,img2,...,img28,img29,img30,img31,img32,img33,img34,img35,img36,img37
0,34.0,46,18,66,23,9,33,ses1,6.081467,8.853849,...,11.229273,7.994739,8.891150,7.892396,7.583052,11.331930,7.948770,8.425824,6.491228,7.228966
1,NaN,102,62,46,51,31,23,NaN,16.610891,12.185616,...,21.496401,24.417400,24.012934,17.057951,21.242817,19.575087,24.481674,22.414888,23.080372,24.868599
2,NaN,50,12,62,25,6,31,NaN,3.952931,5.268596,...,5.830733,4.205112,6.116723,4.504898,5.164454,5.174181,3.034716,3.246057,3.203902,3.446852
3,NaN,40,56,46,20,28,23,NaN,14.523890,22.689468,...,20.756020,16.112068,25.341473,12.902251,26.214119,18.599001,18.485971,17.540710,19.565052,12.754345
4,NaN,60,20,50,30,10,25,NaN,5.321487,7.304843,...,6.473924,4.850379,7.060949,6.028605,5.067628,5.088174,4.733837,5.295754,2.696215,4.582192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,NaN,46,16,66,23,8,33,NaN,6.257287,7.567688,...,2.262680,6.649792,8.483969,7.583700,5.432410,4.880533,8.306602,3.687881,5.573971,4.854362
3605,NaN,94,62,48,47,31,24,NaN,8.060211,10.215308,...,3.041367,5.804007,11.560439,10.979113,6.780633,7.086624,7.057034,1.510613,12.391757,7.078077
3606,NaN,84,26,40,42,13,20,NaN,10.409855,13.369532,...,3.079352,13.493482,8.450074,10.054090,3.681234,8.652498,9.823444,5.281919,8.985109,13.855386
3607,NaN,102,14,58,51,7,29,NaN,7.803307,15.727143,...,2.596920,14.724689,13.228936,13.131496,9.166053,10.984136,13.817241,4.601292,13.005678,6.713248


In [6]:
lst1 = in_df['vx']
lst2 = in_df['vy']
lst3 = in_df['vz']
len(lst3)

3609

In [7]:
combined_lst = []
for i in range(len(lst1)):
    c = str(lst1[i]) + " " + str(lst2[i]) + " " + str(lst3[i])
    combined_lst.append(c)
    
length = len(combined_lst)
length

3609

In [8]:
# combined_lst = list(set(combined_lst))
# len(combined_lst)

In [9]:
cols = []
for i in range(0,37):
    col = 'img' + str(i+1)
    cols.append(col)

In [10]:
comp_df = pd.DataFrame(columns=cols)
comp_df

,img1,img2,img3,img4,img5,img6,img7,img8,img9,img10,...,img28,img29,img30,img31,img32,img33,img34,img35,img36,img37


In [11]:
output_df = comp_df.copy()
output_df

,img1,img2,img3,img4,img5,img6,img7,img8,img9,img10,...,img28,img29,img30,img31,img32,img33,img34,img35,img36,img37


In [12]:
in_df['C']

0       34.0
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
3604     NaN
3605     NaN
3606     NaN
3607     NaN
3608     NaN
Name: C, Length: 3609, dtype: float64

In [13]:
clust_lst = in_df['C']
clust_arr = np.array(clust_lst, dtype=np.float64)

In [14]:
np.isnan(clust_lst[2])

True

In [15]:
index_lst = []
for i in range(len(clust_lst)):
    if np.isnan(clust_lst[i]) == False:
        index_lst.append(i)

print(len(index_lst))
print(len(index_lst[1:]))
index_lst = index_lst[1:]
print(len(index_lst))

82
81
81


In [16]:
Final_outputs = []

In [17]:
out_num = out_df.to_numpy()

In [18]:
len(out_num)

3034

In [19]:
len(comp_df.columns)

37

In [20]:
out_num

array([[0],
       [0],
       [1],
       ...,
       [0],
       [2],
       [0]], dtype=int64)

In [21]:
Final_outputs = []
out_ind = 0
ind = 0
j = 0
lst_start = 0
while ind < length:
    if j < len(index_lst) and ind == index_lst[j]:
        lst_start += 37
        j += 1
    
    row = in_df.iloc[ind]
    
    imagenet = []
    coco = []
    sun = []
    
    detrend_in = []
    for pos in range(0,37):
        name = "img" + str(pos+1)
        detrend_in.append(float(row[name]))
        
    detrend_out = signal.detrend(np.array(detrend_in))
    
    coco_pres = False
    imagenet_pres = False
    sun_pres = False
    
    row_inputs = detrend_out.tolist()
    row_outputs = []
    
    for pos in range(0,37):
        out_ind = lst_start + pos
        if out_num[out_ind][0] == 0:
            row_outputs.append(0)
            
        elif out_num[out_ind][0] == 1:
            row_outputs.append(1)
        
        elif out_num[out_ind][0] == 2:
            row_outputs.append(2)
        
    inp_data_to_append = {}
    out_data_to_append = {}
    for i in range(len(comp_df.columns)):
        inp_data_to_append[comp_df.columns[i]] = row_inputs[i]
        out_data_to_append[output_df.columns[i]] = row_outputs[i]

    comp_df = comp_df.append(inp_data_to_append, ignore_index = True)
    output_df = output_df.append(out_data_to_append, ignore_index = True)
            
    
    ind += 1
    
comp_df

,img1,img2,img3,img4,img5,img6,img7,img8,img9,img10,...,img28,img29,img30,img31,img32,img33,img34,img35,img36,img37
0,-1.239142,1.498891,-0.473891,2.732457,-1.775369,2.018271,0.878624,0.189638,-3.116104,0.704959,...,2.981228,-0.287656,0.574405,-0.458697,-0.802391,2.912138,-0.505372,-0.062667,-2.031613,-1.328224
1,2.574738,-2.062863,5.398529,7.057920,-6.506147,-0.266727,2.404262,3.955059,-2.694261,0.519040,...,1.727434,4.436107,3.819314,-3.347995,0.624544,-1.255513,3.438749,1.159636,1.612793,3.188694
2,-0.349369,0.966855,-0.973846,1.972532,-1.548146,2.045534,-0.396231,-1.012574,-0.740672,0.159815,...,1.543536,-0.081525,1.830645,0.219379,0.879495,0.889782,-1.249124,-1.037224,-1.078819,-0.835310
3,-2.361752,5.785539,-0.705231,10.028336,-4.037800,0.293847,-0.193063,-2.664406,0.030713,0.171018,...,3.376617,-1.285622,7.925495,-4.532014,8.761567,1.128161,0.996844,0.033296,2.039350,-4.789645
4,-0.013275,1.999355,0.181871,1.926223,0.408957,2.354265,0.389001,-2.391864,-1.995376,0.866367,...,1.929570,0.335299,2.575144,1.572074,0.640372,0.690192,0.365129,0.956321,-1.613944,0.301307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,0.846497,2.147211,2.059639,-3.180419,-1.287810,0.233190,-2.042279,-1.245020,-0.205599,0.766990,...,-3.409642,0.967784,2.792274,1.882319,-0.278658,-0.840221,2.576162,-2.052246,-0.175842,-0.905137
3605,1.646910,3.769119,2.164775,-3.303450,0.056773,1.809287,-3.074306,-2.480639,-2.745012,-3.757234,...,-4.259931,-1.530180,4.193364,3.579149,-0.652219,-0.379118,-0.441596,-6.020906,4.827350,-0.519219
3606,2.006820,4.994503,3.825468,-6.685461,-1.013922,3.145338,-3.431581,1.846257,4.234724,4.299337,...,-4.567513,5.874623,0.859222,2.491244,-3.853606,1.145665,2.344617,-2.168902,1.562295,6.460578
3607,-1.856298,6.040045,3.044136,-8.275523,-3.336843,-1.374764,-6.508755,0.523651,3.207984,1.256430,...,-7.805001,4.295275,2.772028,2.647095,-1.345841,0.444748,3.250360,-5.993082,2.383811,-3.936112


In [22]:
output_df

,img1,img2,img3,img4,img5,img6,img7,img8,img9,img10,...,img28,img29,img30,img31,img32,img33,img34,img35,img36,img37
0,0,0,1,2,1,1,2,0,2,2,...,2,2,0,0,2,1,1,1,0,0
1,0,0,1,2,1,1,2,0,2,2,...,2,2,0,0,2,1,1,1,0,0
2,0,0,1,2,1,1,2,0,2,2,...,2,2,0,0,2,1,1,1,0,0
3,0,0,1,2,1,1,2,0,2,2,...,2,2,0,0,2,1,1,1,0,0
4,0,0,1,2,1,1,2,0,2,2,...,2,2,0,0,2,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,1,2,1,0,0,1,0,1,0,1,...,1,1,1,1,1,0,1,0,2,0
3605,1,2,1,0,0,1,0,1,0,1,...,1,1,1,1,1,0,1,0,2,0
3606,1,2,1,0,0,1,0,1,0,1,...,1,1,1,1,1,0,1,0,2,0
3607,1,2,1,0,0,1,0,1,0,1,...,1,1,1,1,1,0,1,0,2,0


In [23]:
comp_df.to_csv("../Data/Final_data/Final_inputs_4.csv", index = False)
output_df.to_csv("../Data/Final_data/Final_outputs_4.csv", index = False)